Import libraries

In [41]:
import tensorflow as tf
from keras.utils import image_dataset_from_directory

: 

# Part 1 - Data Preprocessing

preprocessing the training set

In [42]:
# I found that Keras had depracated the ImageDataGenerator class. A few minutes of research revealed that had been replaced with "image_dataset_from_directory" function.

# Load training dataset from directory
training_set = tf.keras.utils.image_dataset_from_directory(
   'C:/Users/henry/Desktop/Personal/Training/Project/Medical-X-Ray-Image-Classifier-Pneumonia-Detection/chest_xray/train',
    labels='inferred',                 # Infer labels based on subdirectory names
    label_mode='binary',               # Binary classification
    image_size=(64, 64),               # Resize all images to 64x64
    batch_size=32,                     # Batch size of 32
    shuffle=True                       # Shuffle dataset
)

# Data augmentation layers (to replace ImageDataGenerator transformations)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),         # Rescale pixel values to [0, 1], this applies feature scaling to each pixel by dividing their value by 255. we already know that each pixel takes a value of 255, so by dividing all the pixels by 255, we get all pixels values between 0 and 1 which is just like a normalization.
    tf.keras.layers.RandomFlip("horizontal"),  # Apply horizontal flipping
    tf.keras.layers.RandomRotation(0.2),       # Simulate shear transformations
    tf.keras.layers.RandomZoom(0.2)            # Apply zoom transformations
])


Found 5216 files belonging to 2 classes.


preprocessing the test set

In [43]:
# Load the test dataset
test_set = tf.keras.utils.image_dataset_from_directory(
    'C:/Users/henry/Desktop/Personal/Training/Project/Medical-X-Ray-Image-Classifier-Pneumonia-Detection/chest_xray/test',
    labels='inferred',                 # Infer labels from subdirectory names
    label_mode='binary',               # Binary classification
    image_size=(64, 64),               # Resize all images to 64x64
    batch_size=32,                     # Batch size of 32
    shuffle=False                      # No shuffling for test dataset
)

# Apply rescaling to normalize pixel values (equivalent to rescale=1./255)
normalization_layer = tf.keras.layers.Rescaling(1./255)

Found 624 files belonging to 2 classes.


# Part 2 - Building the CNN

Initialising the CNN

In [44]:
cnn = tf.keras.Sequential()

: 

Step 1 - Convolution

In [45]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Step 2 - Pooling

In [46]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second layers to the CNN

In [47]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [48]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [49]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [50]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

Compiling the CNN

In [51]:
cnn.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the CNN

In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

In [53]:
# Part 4 - Making a Single Prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Define the test directory (parent folder containing NORMAL and PNEUMONIA)
test_dir = "C:/Users/henry/Desktop/Personal/Training/Project/Medical-X-Ray-Image-Classifier-Pneumonia-Detection/chest_xray/test"

# Ask the user for an image name
image_name = input("Enter the image file name (including extension, e.g., person62_virus_119.jpeg): ")

# Search for the image inside both NORMAL and PNEUMONIA folders
image_path = None
for root, dirs, files in os.walk(test_dir):
    if image_name in files:
        image_path = os.path.join(root, image_name)
        break  # Stop searching once the image is found

# Check if the image was found
if image_path is None:
    print("Error: Image not found in the test dataset!")
    exit()
else:
    print(f"Selected Image: {image_path}")

    # Load and preprocess the image
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)  # Expand dimensions for model input
    test_image /= 255.0  # Normalize pixel values

    # Predict using the trained model
    result = cnn.predict(test_image)

    # Define class indices
    class_indices = {'NORMAL': 0, 'PNEUMONIA': 1}
    class_labels = {v: k for k, v in class_indices.items()}  # Reverse mapping

    # Get predicted class
    predicted_class_index = 1 if result[0][0] > 0.5 else 0  # If model outputs probabilities, use argmax
    prediction = class_labels[predicted_class_index]

    print(f"Predicted Class: {prediction}")
